# Clean Raw Data

in this '.ipynb' I clean the raw fine data and extract what is needed to be exctracted

In [4]:
import numpy as np
import pandas as pd
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
NumPy version: 1.26.4
Pandas version: 2.2.3


In [13]:
RAW_DATA = "../data/raw_data/fine_data_raw.csv"

df = pd.read_csv(RAW_DATA, delimiter=';')
df.shape

/var/folders/vc/9vc1pfds1jq7ck20388nlx0h0000gn/T/ipykernel_54486/3460472636.py:3: DtypeWarning: Columns (16,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA, delimiter=';')


(502451, 25)

In [14]:
df.head()



,FineType,Agency,ViolationDate,LicenePlateTemplate,IsCustom,ReceivedAt,FinalDiscountDate,FinalPaymentDate,OriginalAmount,FinalAmount,...,CityCoords,WasTowed,TowedFromCoords,TowingEndDate,TowingFee,TowingStartDate,Column1,Column2,Column3,CameraCoords
0,აა000000,Police,22.02.15 00:00,XX - 000 - XX,f,22.02.15 00:00,NaN,NaN,0,0,...,"41.9208048, 41.9931328",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ეთ000000,Police,10.01.16 14:27,XX - 000 - XX,f,10.01.16 14:27,NaN,09.02.16 00:00,1000,3000,...,"41.732575, 44.796886",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ეი000000,Police,11.08.16 00:00,XX - 000 - XX,f,11.08.16 00:00,NaN,NaN,5000,15000,...,"42.0902225, 42.0359712",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ეი000000,Police,20.01.17 23:48,XX - 000 - XX,f,20.01.17 23:48,NaN,NaN,100000,150000,...,"41.6723515, 44.9136321",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ეი000000,Police,16.02.17 00:00,XX - 000 - XX,f,16.02.17 00:00,NaN,20.03.17 00:00,20000,60000,...,"41.7151151, 44.8162896",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
import pandas as pd

def extract_camera_capabilities(df):
    # Initialize list to store camera data
    camera_data = []
    
    # Group by camera coordinates
    grouped = df.groupby('CameraCoords')
    
    for coords, group in grouped:
        # Skip if coordinates are NaN
        if pd.isna(coords) or coords == 'NaN':
            continue
            
        # Split coordinates and preserve all digits
        try:
            coord_x, coord_y = coords.split(',')
            # Strip whitespace but keep as string to preserve precision
            coord_x = coord_x.strip()
            coord_y = coord_y.strip()
            # Convert to float for validation but keep original string
            float(coord_x), float(coord_y)  # This just validates they're valid numbers
        except:
            continue
            
        # Get unique articles (violation codes) for this camera
        unique_articles = sorted(group['Article'].unique().tolist())
        
        # Create entry for this camera
        camera_entry = {
            'coord_x': coord_x,  # Keep as string to preserve precision
            'coord_y': coord_y,  # Keep as string to preserve precision
            'articles': unique_articles,
            'article_count': len(unique_articles)
        }
        
        camera_data.append(camera_entry)
    
    # Create new DataFrame from processed data
    camera_df = pd.DataFrame(camera_data)
    
    return camera_df

# Usage:
# camera_df = extract_camera_capabilities(your_original_df)

In [34]:
import pandas as pd
import numpy as np

def extract_camera_capabilities(df):
    # Initialize list to store camera data
    camera_data = []
    
    # Group by camera coordinates
    grouped = df.groupby('CameraCoords')
    
    for coords, group in grouped:
        # Skip if coordinates are NaN
        if pd.isna(coords) or coords == 'NaN':
            continue
            
        # Split coordinates and preserve all digits
        try:
            coord_x, coord_y = coords.split(',')
            # Strip whitespace but keep as string to preserve precision
            coord_x = coord_x.strip()
            coord_y = coord_y.strip()
            # Convert to float for validation but keep original string
            float(coord_x), float(coord_y)  # This just validates they're valid numbers
        except:
            continue
            
        # Get unique articles (violation codes) for this camera
        unique_articles = sorted(group['Article'].unique().tolist())
        
        # Create entry for this camera
        camera_entry = {
            'coord_x': coord_x,  # Keep as string to preserve precision
            'coord_y': coord_y,  # Keep as string to preserve precision
            'articles': unique_articles,
            'article_count': len(unique_articles)  # Add count for easy stats
        }
        
        camera_data.append(camera_entry)
    
    # Create new DataFrame from processed data
    camera_df = pd.DataFrame(camera_data)
    
    return camera_df

# Assuming your original DataFrame is called 'df'
camera_capabilities_df = extract_camera_capabilities(df)

# Calculate and display statistics
print("\n=== Camera Network Statistics ===")
print(f"Total unique cameras: {len(camera_capabilities_df)}")
print(f"\nArticle Detection Capabilities:")
print(f"- Maximum articles per camera: {camera_capabilities_df['article_count'].max()}")
print(f"- Minimum articles per camera: {camera_capabilities_df['article_count'].min()}")
print(f"- Average articles per camera: {camera_capabilities_df['article_count'].mean():.2f}")
print(f"- Median articles per camera: {camera_capabilities_df['article_count'].median()}")

# Article frequency analysis
all_articles = [article for articles in camera_capabilities_df['articles'] for article in articles]
article_freq = pd.Series(all_articles).value_counts()

print(f"\nMost Common Article Types:")
print(article_freq.head())

# Camera capability distribution
capability_groups = camera_capabilities_df['article_count'].value_counts().sort_index()
print(f"\nCamera Capability Distribution:")
for count, num_cameras in capability_groups.items():
    print(f"Cameras with {count} article types: {num_cameras}")

# View the first few rows
camera_capabilities_df.head()


=== Camera Network Statistics ===
Total unique cameras: 609

Article Detection Capabilities:
- Maximum articles per camera: 13
- Minimum articles per camera: 1
- Average articles per camera: 3.35
- Median articles per camera: 2.0

Most Common Article Types:
125-1      380
118-2-1    278
125-1-1    198
118-3-1    192
118-4-1    173
Name: count, dtype: int64

Camera Capability Distribution:
Cameras with 1 article types: 178
Cameras with 2 article types: 152
Cameras with 3 article types: 67
Cameras with 4 article types: 34
Cameras with 5 article types: 43
Cameras with 6 article types: 41
Cameras with 7 article types: 42
Cameras with 8 article types: 23
Cameras with 9 article types: 15
Cameras with 10 article types: 10
Cameras with 11 article types: 2
Cameras with 12 article types: 1
Cameras with 13 article types: 1


,coord_x,coord_y,articles,article_count
0,41.27161802314097,43.5852946686558,"[118-2-1, 118-2-2, 118-3-1, 118-4-1]",4
1,41.33615812164436,44.222693015546284,"[118-2-1, 118-3-1]",2
2,41.404476968498834,43.48819842391187,[118-2-1],1
3,41.44461050342212,44.6385143335216,[125-1],1
4,41.44519707590954,44.60569665468411,[125-1],1


In [36]:
import json

def export_to_typescript(df, output_file='camera_data.ts'):
    # Convert DataFrame to list of dictionaries
    camera_data = df.to_dict('records')
    
    # Create TypeScript content
    ts_content = """// Auto-generated camera capabilities data
export interface CameraCapability {
    coord_x: string;
    coord_y: string;
    articles: string[];
    article_count: number;
}

export const cameras: CameraCapability[] = %s;
""" % json.dumps(camera_data, indent=2, ensure_ascii=False)
    
    # Write to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(ts_content)
    
    # Print stats
    print(f"Exported {len(camera_data)} cameras to {output_file}")
    print(f"Total unique articles: {len(set(article for cam in camera_data for article in cam['articles']))}")

# Usage:
# Assuming your DataFrame is called camera_capabilities_df:
export_to_typescript(camera_capabilities_df)

Exported 609 cameras to camera_data.ts
Total unique articles: 31
